# Transfer Learning

In [24]:
import tensorflow_hub as hub
import tensorflow as tf
import tf_keras
import numpy as np
import os
import tensorflow_datasets as tfds
print("Version: ", tf.__version__)
print("Hub version: ", hub.__version__)

Version:  2.17.0
Hub version:  0.16.1


data_dir = 'aclImdb'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# batch_size = 32
seed = 42
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir,
    
    seed = seed,
    subset = "training",
    validation_split = 0.4
)

raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir,
    
    seed = seed,
    subset = "validation",
    validation_split = 0.4
)

raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    test_dir,
    
    
)

In [25]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [26]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [27]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [28]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [29]:
model = tf_keras.Sequential()
model.add(hub_layer)
model.add(tf_keras.layers.Dense(16, activation='relu'))
model.add(tf_keras.layers.Dense(1))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_4 (Dense)             (None, 16)                816       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48191433 (183.84 MB)
Trainable params: 48191433 (183.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
model.compile(optimizer= 'adam', loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [31]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs=20, validation_data=validation_data.batch(512), verbose=1)

Epoch 1/20
30/30 [==============================] - 31s 955ms/step - loss: 0.7021 - accuracy: 0.4975 - val_loss: 0.6458 - val_accuracy: 0.5259
Epoch 2/20
30/30 [==============================] - 25s 816ms/step - loss: 0.5852 - accuracy: 0.6057 - val_loss: 0.5442 - val_accuracy: 0.6808
Epoch 3/20
30/30 [==============================] - 24s 807ms/step - loss: 0.4596 - accuracy: 0.7716 - val_loss: 0.4446 - val_accuracy: 0.7756
Epoch 4/20
30/30 [==============================] - 24s 805ms/step - loss: 0.3409 - accuracy: 0.8641 - val_loss: 0.3726 - val_accuracy: 0.8296
Epoch 5/20
30/30 [==============================] - 24s 812ms/step - loss: 0.2509 - accuracy: 0.9071 - val_loss: 0.3333 - val_accuracy: 0.8531
Epoch 6/20
30/30 [==============================] - 24s 811ms/step - loss: 0.1860 - accuracy: 0.9391 - val_loss: 0.3146 - val_accuracy: 0.8630
Epoch 7/20
30/30 [==============================] - 25s 824ms/step - loss: 0.1386 - accuracy: 0.9594 - val_loss: 0.3081 - val_accuracy: 0.8691

In [33]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 4s - loss: 0.4663 - accuracy: 0.8472 - 4s/epoch - 73ms/step
loss: 0.466
accuracy: 0.847
